In [3]:
import iris
import iris.plot as iplt
import iris.quickplot as qplt
import iris.analysis
import iris.coord_categorisation
import iris.plot
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math # Use to call euler's number (e)
import os
from calendar import monthrange
from tqdm import tqdm
import concurrent.futures
from concurrent.futures import ProcessPoolExecutor
from multiprocess import Pool
%run Lu_and_Romps.py
from Lu_and_Romps_vectorized import heat_index_lu_and_romps_vectorized

from iris.util import mask_cube_from_shapefile

In [2]:
def set_permission(path, mode=0o777):
    '''
    Give a filename(s) permissions

    Inputs:
    filename - a string of the filename
    mode - permission mode (default=0777)n
    '''
    try:
        os.chmod(path, mode)
    except OSError:
        print("Cannot change permissions: {}".format(path))

def save_cube(cube, filename):
    iris.save(cube, filename)
    set_permission(filename)
    print('Saved {}'.format(filename))

# ---------------------------------------------------------------------------------------------------------------------------- #
                                                # Code From Catherine 17/06/2024
# ---------------------------------------------------------------------------------------------------------------------------- #

def get_analysis_method(stat):
    '''
    returns the iris analysis method from a string  

    Input Arguments:
    ---------------
    * stat
        string of the stat type to use (min, mean, max, sum, stdev or steddev)
    
    Returns:
    -------
    * analysis_method
        iris analysis method for the requested statistic
    '''
    if stat == 'mean':
        analysis_method = iris.analysis.MEAN
    elif stat == 'min':
        analysis_method = iris.analysis.MIN
    elif stat == 'max':
        analysis_method = iris.analysis.MAX
    elif stat == 'sum':
        analysis_method = iris.analysis.SUM
    elif stat in ['stddev', 'stdev']:
        analysis_method = iris.analysis.STD_DEV
    elif stat in ['percentile']:
        analysis_method = iris.analysis.PERCENTILE
    else:
        analysis_method = None
    return analysis_method
 
def get_spatial_ave_cube(cube, spatial_stat):
    '''
    Calculates the spatial statistic from a cube, e.g. the area mean
   
    Input Arguments:
    ---------------
    * cube
        cube to do the calculation over
    * spatial_stat
        string of the stat type to use in the spatial domain (min, mean, max, sum, stdev or stddev)
  
    Returns:
    -------
    * cube
        cube of the spatial statistic     

    '''
    spatial_analysis_method = get_analysis_method(spatial_stat)
    latitude_coord, longitude_coord = get_cube_lat_lon_coords(cube)
    try:
        cube.coord(latitude_coord).guess_bounds()
    except:
        pass
    try:
        cube.coord(longitude_coord).guess_bounds()
    except:
        pass
    if latitude_coord != 'latitude' and longitude_coord != 'longitude':
        cube = cube.collapsed([latitude_coord, longitude_coord], spatial_analysis_method)
    else:
        cube_grid_areas = iris.analysis.cartography.area_weights(cube)
        cube = cube.collapsed([latitude_coord, longitude_coord], spatial_analysis_method, weights=cube_grid_areas)
    return cube

 
def add_year_dim(cube):
    '''A function that returns a cube with the year attached as a dimension
 
    Input args:
    ----------
    cube  -- the cube to add the year to
    '''
    try:
        cube.coord('year')
    except:
        iris.coord_categorisation.add_year(cube, 'time')
    return cube

def get_year_of_temporal_statistic(cube, temporal_stat, year_season_year='year'):
    '''
    Calculates the overall temporal statistic from a cube based on
    the input statistics and returns the year in which this happens.
    E.g. the maximum (temporal) value. Expects a 1-D cube   

    Input Arguments:
    ---------------
    * cube
        cube to do the calculation over
    * temporal_stat
        string of the stat type to use in the temporal domain (min, mean, max, sum, stdev or stddev)  
    Returns:
    -------
    * year
        integer of the year for the requested statistic
    '''
    temporal_analysis_method = get_analysis_method(temporal_stat)
    value = cube.collapsed(['time'], temporal_analysis_method)
    value = value.data
    itemindex = np.where(cube.data==value)
    if year_season_year == 'year':
        cube = add_year_dim(cube)
        year = cube.coord('year').points[itemindex]
    elif year_season_year == 'season_year':
        cube = add_season_year_dim(cube)
        year = cube.coord('season_year').points[itemindex]
    return year

def add_month_dim(cube):
    '''A function that returns a cube with the month attached as a
    dimension

    Input args:
    ----------
    cube  -- the cube to add the month to
    '''
    try:
        cube.remove_coord('month')
    except:
        pass
    iris.coord_categorisation.add_month(cube, 'time', name='month')
    return cube

def add_year_dim(cube):
    '''A function that returns a cube with the year attached as a
    dimension

    Input args:
    ----------
    cube  -- the cube to add the year to
    '''
    try:
        cube.remove_coord('year')
    except:
        pass
    iris.coord_categorisation.add_year(cube, 'time', name='year')
    return cube

def subset_cube_by_season(cube, season):
    '''A function that returns a subset of a cube based on input season
    Input args:
    ----------
    cube  -- the cube to be subset
    mon   -- the month number to subset by
    '''
    try:
        add_season_dim(cube)
    except:
        pass
    subset_cube = cube.extract(iris.Constraint(season=season))
    return subset_cube
 
def get_cube_lat_lon_coords(cube):
    '''A function that returns the latitude and longitude dimensions of the

    input cube

    Input args:
    ----------
    cube  - the cube to be manipulated 

    Returns:
    -------
    latitude_coord - the latitude coordinate
    longitude_coord - the longitude coordinate
    '''
    cube_coords = ascend.shape.cube_primary_xy_coord_names(cube) # CHANGE TO WORK AS CAN'T ACCESS ASCEND
    latitude_coord = cube_coords[0]
    longitude_coord = cube_coords[1]
    return latitude_coord, longitude_coord

def extract_yearly_cubes(cube):
    yearly_cubes = []
    years = set([cell.point.year for cell in cube.coord('time').cells()])
    for year in years:
        year_constraint = iris.Constraint(time=lambda cell: cell.point.year == year)
        year_cube = cube.extract(year_constraint)
        yearly_cubes.append(year_cube)
    iris.cube.CubeList(yearly_cubes)
    return yearly_cubes

def extract_monthly_cubes(cube):
    monthly_cubes = []
    # Extract unique years
    years = set([cell.point.year for cell in cube.coord('time').cells()])
    # Iterate over each year
    for year in sorted(years):  # Sorting to ensure chronological order
        # Iterate over each month
        for month in range(1, 13):
            # Apply constraint for specific year and month
            month_constraint = iris.Constraint(time=lambda cell: cell.point.year == year and cell.point.month == month)
            month_cube = cube.extract(month_constraint)
            if month_cube:  # Check if the cube is not empty
                monthly_cubes.append(month_cube)
    return monthly_cubes

def extract_daily_cubes(cube):
    daily_cubes = []
    # Extract unique years
    years = set([cell.point.year for cell in cube.coord('time').cells()])
    # Iterate over each year
    for year in sorted(years):  # Sorting to ensure chronological order
        # Iterate over each month
        for month in range(1, 13):
            # Determine the number of days in the month
            days_in_month = monthrange(year, month)[1]
            # Iterate over each day
            for day in range(1, days_in_month + 1):
                # Apply constraint for specific year, month, and day
                day_constraint = iris.Constraint(time=lambda cell: cell.point.year == year and cell.point.month == month and cell.point.day == day)
                day_cube = cube.extract(day_constraint)
                if day_cube:  # Check if the cube is not empty
                    daily_cubes.append(day_cube)
    return daily_cubes

# ================================================ # My Code
def calculate_absolute_vapour_pressure(location_name, td_cube):
    """ Calculate absolute vapour pressure array
        Parameters:
            location_name: name of location (string)
            td: dew point temperature (cube object)
        Returns:
            cube object
    """
    filename = (f'{location_name}_es_hourly_{start_year}-{end_year}.nc') ## IF NOT WORKING: it's because I added "es" before "hourly"

    # Check if the file already exists
    if os.path.isfile(filename):
        es_cube = iris.load_cube(filename)
        print('es_cube already exists. Skipping es_cube calculation')
    else:
        print('creating es_cube')
        es_cube = td_cube.copy()
        
        es_cube.data = (0.6108 * np.exp(17.27 * td_cube.data / (237.3 + td_cube.data)))

        es_cube.data.fill_value = -9999
        es_cube = add_year_dim(es_cube) # Adds a year dimension to the cube
        es_cube = add_month_dim(es_cube) # Adds a month dimension to the cube

        es_cube.long_name = "absolute vapour pressure"
        es_cube.var_name = None
        es_cube.units = "kPa"
        save_cube(es_cube, filename)
        print(f"es_cube saved as '{location_name}_es_hourly_{start_year}-{end_year}.nc'")

    return es_cube

def calculate_saturation_vapour_pressure(location_name, tas_cube):
    """ Calculate saturation vapour pressure array
        Parameters:
            location_name: name of location (string)
            tas: temperature at surface (cube object)
        Returns:
            cube object
    """
    filename = (f'{location_name}_saturation_vapour_pressure_hourly_{start_year}-{end_year}.nc')
    
    # Check if the file already exists
    if os.path.isfile(filename):
        saturation_vapour_pressure_cube = iris.load_cube(filename)
        print('saturation_vapour_pressure_cube already exists. Skipping saturation_vapour_pressure_cube calculation.')
    else:
        print('creating saturation_vapour_pressure_cube')
        saturation_vapour_pressure_cube = tas_cube.copy()

        saturation_vapour_pressure_cube.data = (0.6108 * np.exp(17.27 * tas_cube.data / (237.3 + tas_cube.data)))
        saturation_vapour_pressure_cube.long_name = "saturation vapour pressure"
        saturation_vapour_pressure_cube.var_name = None
        saturation_vapour_pressure_cube.units = "kPa"

        save_cube(saturation_vapour_pressure_cube, filename)
        print(f'saturation_vapour_pressure_cube saved as {filename}')

    return saturation_vapour_pressure_cube

def calculate_relative_humidity(location_name, tas_cube, saturation_vapour_pressure_cube):
    """ Calculate relative humidity array
        Parameters:
            location_name: name of location (string)
            tas: temperature at surface (cube object)
            saturation_vapour_pressure: saturation vapour pressure (cube object)
        Returns:
            cube object
    """
    filename = (f'{location_name}_rh_hourly_{start_year}-{end_year}.nc')

    # Check if the file already exists
    if os.path.isfile(filename):
        rh_cube = iris.load_cube(f'{location_name}_rh_hourly_{start_year}-{end_year}.nc')   
        print('rh_cube already exists. Skipping rh_cube calculation')
    else:
        rh_cube = tas_cube.copy()
        rh_cube.data = (es_cube.data / saturation_vapour_pressure_cube.data) * 100
        rh_cube.data.fill_value = -9999
        rh_cube = add_year_dim(rh_cube) # Adds a year dimension to the cube
        rh_cube = add_month_dim(rh_cube) # Adds a month dimension to the cube

        rh_cube.long_name = "relative humidity"
        rh_cube.var_name = "RH"
        rh_cube.units = "%"

        filename = (f'{location_name}_rh_hourly_{start_year}-{end_year}.nc')
        save_cube(rh_cube, filename)
        print(f"rh_cube saved as '{location_name}_rh_hourly_{start_year}-{end_year}.nc'")

    return rh_cube

def calculate_monthly_average_heat_index(tas_cube, rh_cube):
    # Step 1: Extract monthly cubes
    tas_monthly_cubes = extract_monthly_cubes(tas_cube)
    rh_monthly_cubes = extract_monthly_cubes(rh_cube)
    
    monthly_heat_indexes = []
    
    # Assuming tas_monthly_cubes and rh_monthly_cubes are lists of cubes for each month
    for tas_month_cube, rh_month_cube in zip(tas_monthly_cubes.slices_over('time'), rh_monthly_cubes.slices_over('time')):
        # Step 2 & 3: Calculate heat index for each month
        heat_index_monthly = heat_index_lu_and_romps_vectorized(tas_month_cube, rh_month_cube)
        
        # Create a new cube for the heat index with the same metadata as tas_month_cube
        heat_index_cube = tas_month_cube.copy(data=heat_index_monthly)
        
        monthly_heat_indexes.append(heat_index_cube)
    
    # Step 4: Aggregate if necessary or return the list of monthly heat index cubes
    return monthly_heat_indexes

In [10]:
import os
import iris

def create_my_files():
    global tas_cube, rh_cube, es_cube, td_cube
    # Creating the tas_cube
    if os.path.isfile(f'{location_name}_tas_hourly_{start_year}-{end_year}.nc'):
        tas_cube = iris.load_cube(f'{location_name}_tas_hourly_{start_year}-{end_year}.nc')
        print('tas_cube already exists. Skipping tas_cube calculation.')
    else:
        print('creating tas_cube')
        tas_cube = iris.load_cube(filepaths['tas'])

        tas_cube.convert_units('celsius')
        tas_cube.data.fill_value = -9999
        tas_cube = add_year_dim(tas_cube) # Adds a year dimension to the cube
        tas_cube = add_month_dim(tas_cube) # Adds a month dimension to the cube

        iris.save(tas_cube, f'{location_name}_tas_hourly_{start_year}-{end_year}.nc')
        print(f'tas_cube saved as {location_name}_tas_hourly_{start_year}-{end_year}.nc')
    
        
    # Save and delete tas_cube to free memory
    iris.save(tas_cube, f'{location_name}_tas_hourly_{start_year}-{end_year}.nc')

    # Creating the td_cube
    if os.path.isfile(f'{location_name}_td_hourly_{start_year}-{end_year}.nc'):
        td_cube = iris.load_cube(f'{location_name}_td_hourly_{start_year}-{end_year}.nc')
        print('td_cube already exists. Skipping td_cube calculation.')
    else:
        print('creating td_cube')
        td_cube = iris.load_cube(filepaths['td'])
        
        td_cube.convert_units('celsius')
        td_cube.data.fill_value = -9999
        td_cube = add_year_dim(td_cube) # Adds a year dimension to the cube
        td_cube = add_month_dim(td_cube) # Adds a month dimension to the cube

        iris.save(td_cube, f'{location_name}_td_hourly_{start_year}-{end_year}.nc')

    # Creating the es_cube 
    es_cube = calculate_absolute_vapour_pressure(location_name, td_cube)
    iris.save(es_cube, f'{location_name}_es_hourly_{start_year}-{end_year}.nc')

    saturation_vapour_pressure_cube = calculate_saturation_vapour_pressure(location_name, tas_cube)

    # Creating the rh_cube
    rh_cube = calculate_relative_humidity(location_name, tas_cube, saturation_vapour_pressure_cube)
    iris.save(rh_cube, f'{location_name}_rh_hourly_{start_year}-{end_year}.nc')

    return tas_cube, rh_cube, es_cube, td_cube

NameError: name 'tas_cube' is not defined

### 21 Heat Index Algorithms from Anderson et al. (2013)

In [4]:
### 21 Heat Index Algorithm functions
# Function to calculate Heat Index (HI) using algorithm 1 above
def heat_index_1(location_name, location_tas_cube, location_rh_cube):
    """
    Returns the modified heat index (HI) from a tas and h cube.
    
    Parameters
    ----------
    location_name:
        A string specifying the name of the location of interest
    location_tas_cube :
        An instance of `iris.cube.Cube` representing temperature data in ˚C
    location_rh_cube :
        An instance of `iris.cube.Cube` representing humidity data in %
    """
    assert location_tas_cube.units == 'celsius', "Temperature units must be in celsius"
    assert location_rh_cube.units == '%', 'Ensure humidity units are in %'

    tas_cube_fahrenheit = location_tas_cube.copy()
    tas_cube_fahrenheit.convert_units('fahrenheit')
    tas_data = tas_cube_fahrenheit.data.astype(np.float64)

    humidity_data = location_rh_cube.data.astype(np.float64)


    A = -10.3 + 1.1 * tas_data + 0.047 * humidity_data

    B = (
        -42.379 + 2.04901523 * tas_data + 10.14333127 * humidity_data
        - 0.22475541 * tas_data * humidity_data
        - 6.83783e-3 * tas_data ** 2 - 5.481717e-2 * humidity_data ** 2
        + 1.22874e-3 * tas_data ** 2 * humidity_data
        + 8.5282e-4 * tas_data * humidity_data ** 2
        - 1.99e-6 * tas_data ** 2 * humidity_data ** 2
    )

    hi = np.where(tas_data <= 40, tas_data,
        np.where(A < 79, A,
            np.where((humidity_data < 13) & (80 <= tas_data) & (tas_data <= 112),
                B - ((13 - humidity_data) / 4) * ((17 - (tas_data - 95)) / 17) ** 0.5,
                np.where((humidity_data > 85) & (80 <= tas_data) & (tas_data <= 87),
                    B + 0.02 * (humidity_data - 85) * (87 - tas_data),
                    B
                )
            )
        )
    )

    heat_index_cube = location_tas_cube.copy()
    heat_index_cube.data = (hi-23) * 5/9 # Convert to Celsius
    heat_index_cube.data= np.ma.masked_where(heat_index_cube.data < -10, heat_index_cube.data) # Masking values below -10 that weirdly pop up
    heat_index_cube.long_name = 'Heat Index 1'

    return heat_index_cube

# Function to calculate Heat Index (HI) using algorithm 2 above
def heat_index_2(location_name, location_tas_cube, location_td_cube): # Note: specify the cube, not the cube.data
    """Returns the Schoen (2005) heat index from a tas and td cube.
    
    Parameters
    ----------
    location_tas_cube :
        An instance of :class:`iris.cube.Cube`
    location_td_cube :
        An instance of :class:`iris.cube.Cube`
    """   
    assert location_tas_cube.units == 'celsius' and location_td_cube.units == 'celsius', "Ensure cube units are in celsius"

    tas_cube_copy = location_tas_cube.copy()  # Create a copy of the temperature cube
    td_cube_copy = location_td_cube.copy()  # Create a copy of the td cube

    heat_index_2_data = (
        tas_cube_copy.data
        - 1.0799 * (math.e ** (0.03755 * tas_cube_copy.data))
        * (1 - (math.e ** (0.0801 * (td_cube_copy.data - 14))))
    )

    heat_index_cube = tas_cube_copy.copy()  # Create a new cube with the heat index data
    heat_index_cube.data = heat_index_2_data
    heat_index_cube.long_name = 'Heat Index 2'
    
    return heat_index_cube

# Function to calculate Heat Index (HI) using algorithm 3 above
def heat_index_3(location_name, location_tas_cube, location_td_cube):
    """
    Returns the Schoen (2005) heat index from a tas and td cube.
    
    Parameters
    ----------
    location_tas_cube :
        An instance of `iris.cube.Cube` representing temperature data in ˚C
    location_td_cube :
        An instance of `iris.cube.Cube` representing dew point temperature data in ˚C
    """
    assert location_tas_cube.units == 'celsius' and location_td_cube.units == 'celsius', "Ensure cube units are in celsius"

    tas_cube_fahrenheit = location_tas_cube.copy()
    tas_cube_fahrenheit.convert_units('Fahrenheit') # Convert temperature cube to Fahrenheit
    tas_data = tas_cube_fahrenheit.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time

    td_cube_fahrenheit = location_td_cube.copy()
    td_cube_fahrenheit.convert_units('Fahrenheit') # Convert dew point temperature cube to Fahrenheit
    dc_data = td_cube_fahrenheit.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time

    heat_index_3_data = (
        tas_data
        - 0.9971 * (math.e ** (0.02086 * tas_data))
        * (1 - (math.e ** (0.0445 * (dc_data - 57.2))))
    )

    # Create a new cube with the heat index data
    heat_index_cube = location_tas_cube.copy()
    heat_index_cube.data = (heat_index_3_data - 32) * 5/9
    heat_index_cube.long_name = 'Heat Index 3'

    return heat_index_cube

# Function to calculate Heat Index (HI) using algorithm 4 above
def heat_index_4(location_name, location_tas_cube, location_es_cube):
    """
    Returns the Gaffen and Ross (1999) and Steadman (1984) heat index from a tas and es cube.
    
    Parameters
    ----------
    location_tas_cube :
        An instance of `iris.cube.Cube` representing temperature data in ˚C
    location_es_cube :
        An instance of `iris.cube.Cube` representing water vaour pressure in Kilopascals
    """
    assert location_tas_cube.units == 'celsius', "Ensure temperature units are in celsius"
    assert location_es_cube.units == 'kPa', 'Ensure water vapor pressure units are in Kilopascals'

    tas_cube_copy = location_tas_cube.copy()  # Create a copy of the temperature cube
    es_cube_copy = location_es_cube.copy()  # Create a copy of the es cube

    heat_index_4_data = (
        -1.3 + 0.92 * location_tas_cube.data + 2.2 * location_es_cube.data)

    # Create a new cube with the heat index data
    heat_index_cube = location_tas_cube.copy()
    heat_index_cube.data = heat_index_4_data
    heat_index_cube.long_name = 'Heat Index 4'

    return heat_index_cube

# Function to calculate Heat Index (HI) using algorithm 5 above
def heat_index_5(location_name, location_tas_cube, location_rh_cube):
    """
    Returns the El Morjani et al. (2007) and Oka (2011) heat index from a tas and h cube.
    
    Parameters
    ----------
    location_tas_cube :
        An instance of `iris.cube.Cube` representing temperature data in ˚C
    location_rh_cube :
        An instance of `iris.cube.Cube` representing humidity data in %
    """
    assert location_tas_cube.units == 'celsius', "Ensure temperature units are in celsius"
    assert location_rh_cube.units == '%', 'Ensure humidity units are in %'

    # Create a copy of the temperature cube so when converted it doesn't change the original data
    tas_cube_fahrenheit = location_tas_cube.copy()
    tas_cube_fahrenheit.convert_units('Fahrenheit') # Convert temperature cube to Fahrenheit
    
    tas_data = tas_cube_fahrenheit.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time
    humidity_data = location_rh_cube.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time

    heat_index_5_data = (
        -42.379
        + 2.04901523 * tas_data
        + 10.14333127 * humidity_data
        - 0.22475541 * tas_data * humidity_data
        - (6.83783e-3) * tas_data ** 2
        - (5.481717e-2) * humidity_data ** 2
        + (1.22874e-3) * tas_data ** 2 * humidity_data
        + (8.5282e-4) * tas_data * humidity_data ** 2
        - (1.99e-6) * tas_data ** 2 * humidity_data ** 2
    )

    # Apply correction factor
    correction_mask = np.logical_or(tas_data <= 80, humidity_data <= 40)
    heat_index_5_data[correction_mask] = tas_data[correction_mask]

    # Create a new cube with the heat index data
    heat_index_cube = location_tas_cube.copy()
    heat_index_cube.data = (heat_index_5_data -32) * 5/9
    heat_index_cube.long_name = 'Heat Index 5'
    
    return heat_index_cube
    
# Function to calculate Heat Index (HI) using algorithm 6 above
def heat_index_6(location_name, location_tas_cube, location_rh_cube):
    """
    Returns the Fandoeva et al. (2009) heat index from a tas and h cube.
    
    Parameters
    ----------
    location_tas_cube :
        An instance of `iris.cube.Cube` representing temperature data in ˚C
    location_rh_cube :
        An instance of `iris.cube.Cube` representing humidity data in %
    """
    assert location_tas_cube.units == 'celsius', "Ensure temperature units are in celsius"
    assert location_rh_cube.units == '%', 'Ensure humidity units are in %'

    tas_cube_fahrenheit = location_tas_cube.copy()
    tas_cube_fahrenheit.convert_units('Fahrenheit') # Convert temperature cube to Fahrenheit
    tas_data = tas_cube_fahrenheit.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time
    
    humidity_data = location_rh_cube.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time

    heat_index_6_data = (
        -42.379
        + 2.04901523 * tas_data
        + 10.14333127 * humidity_data
        - 0.22475541 * tas_data * humidity_data
        - (6.83783e-3) * tas_data ** 2
        - (5.481717e-2) * humidity_data ** 2
        + (1.22874e-3) * tas_data ** 2 * humidity_data
        + (8.5282e-4) * tas_data * humidity_data ** 2
        - (1.99e-6) * tas_data ** 2 * humidity_data ** 2
    )

    # Apply correction factor
    correction_mask = np.logical_or(tas_data < 80, humidity_data < 40)
    heat_index_6_data[correction_mask] = tas_data[correction_mask]

    # Create a new cube with the heat index data
    heat_index_cube = location_tas_cube.copy()
    heat_index_cube.data = (heat_index_6_data -32) * 5/9
    heat_index_cube.long_name = 'Heat Index 6'

    return heat_index_cube

# Function to calculate Heat Index (HI) using algorithm 7 above
def heat_index_7(location_name, location_tas_cube, location_rh_cube):
    """
    Returns the Di Cristo et al. (2007) and Rajib et al. (2011) heat index from a tas and h cube.
    
    Parameters
    ----------
    location_tas_cube :
        An instance of `iris.cube.Cube` representing temperature data in ˚C
    location_rh_cube :
        An instance of `iris.cube.Cube` representing humidity data in %
    """
    assert location_tas_cube.units == 'celsius', "Temperature units must be in celsius"
    assert location_rh_cube.units == '%', 'Ensure humidity units are in %'

    tas_cube_fahrenheit = location_tas_cube.copy()
    tas_cube_fahrenheit.convert_units('Fahrenheit') # Convert temperature cube to Fahrenheit
    
    tas_data = tas_cube_fahrenheit.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time
    humidity_data = location_rh_cube.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time

    heat_index_7_data = (
        -42.379
        + 2.04901523 * tas_data
        + 10.14333127 * humidity_data
        - 0.22475541 * tas_data * humidity_data
        - (6.83783e-3) * tas_data ** 2
        - (5.481717e-2) * humidity_data ** 2
        + (1.22874e-3) * tas_data ** 2 * humidity_data
        + (8.5282e-4) * tas_data * humidity_data ** 2
        - (1.99e-6) * tas_data ** 2 * humidity_data ** 2
    )

    # Apply correction factor
    correction_mask = np.logical_or(tas_data <= 78.8, humidity_data <= 39)
    heat_index_7_data[correction_mask] = tas_data[correction_mask]

    # Create a new cube with the heat index data
    heat_index_cube = location_tas_cube.copy()
    heat_index_cube.data = (heat_index_7_data -32) * 5/9
    heat_index_cube.long_name = 'Heat Index 7'

    return heat_index_cube

# Function to calculate Heat Index (HI) using algorithm 8 above
def heat_index_8(location_name, location_tas_cube, location_rh_cube):
    """
    Returns the Johnson and Long (2004) heat index from a tas and h cube.
    
    Parameters
    ----------
    location_tas_cube :
        An instance of `iris.cube.Cube` representing temperature data in ˚C
    location_rh_cube :
        An instance of `iris.cube.Cube` representing humidity data in %
    """
    assert location_tas_cube.units == 'celsius', "Temperature units must be in celsius"
    assert location_rh_cube.units == '%', 'Ensure humidity units are in %'

    tas_cube_fahrenheit = location_tas_cube.copy()
    tas_cube_fahrenheit.convert_units('Fahrenheit') # Convert temperature cube to Fahrenheit
    
    tas_data = tas_cube_fahrenheit.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time
    humidity_data = location_rh_cube.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time

    heat_index_8_data = (
        -42.4
        + 2.049 * tas_data
        + 10.14 * humidity_data
        - 0.2248 * tas_data * humidity_data
        - (6.838e-3) * tas_data ** 2
        - (5.482e-2) * humidity_data ** 2
        + (1.229e-3) * tas_data ** 2 * humidity_data
        + (8.528e-4) * tas_data * humidity_data ** 2
        - (1.99e-6) * tas_data ** 2 * humidity_data ** 2
    )

    # Apply correction factor
    correction_mask = tas_data < 79
    heat_index_8_data[correction_mask] = tas_data.data[correction_mask]

    # Create a new cube with the heat index data
    heat_index_cube = location_tas_cube.copy()
    heat_index_cube.data = (heat_index_8_data -32) * 5/9
    heat_index_cube.long_name = 'Heat Index 8'
    
    return heat_index_cube

# Function to calculate Heat Index (HI) using algorithm 9 above
def heat_index_9(location_name, location_tas_cube, location_rh_cube):
    """
    Returns the Robinson (2001) heat index from a tas and h cube.
    
    Parameters
    ----------
    location_tas_cube :
        An instance of `iris.cube.Cube` representing temperature data in ˚C
    location_rh_cube :
        An instance of `iris.cube.Cube` representing humidity data in %
    """
    assert location_tas_cube.units == 'celsius', "Temperature units must be in celsius"
    assert location_rh_cube.units == '%', 'Ensure humidity units are in %'

    tas_cube_fahrenheit = location_tas_cube.copy()
    tas_cube_fahrenheit.convert_units('Fahrenheit') # Convert temperature cube to Fahrenheit
    
    tas_data = tas_cube_fahrenheit.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time
    humidity_data = location_rh_cube.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time

    heat_index_9_data = (
        16.923
        + 0.185212 * tas_data
        + 5.37941 * humidity_data
        - 0.100254 * tas_data * humidity_data
        + 9.4169e-3 * tas_data ** 2
        + 7.28898e-3 * humidity_data ** 2
        + 3.45372e-4 * tas_data ** 2 * humidity_data
        - 8.14971e-4 * tas_data * humidity_data ** 2
        + 1.02102e-5 * tas_data ** 2 * humidity_data ** 2
        - 3.8646e-5 * tas_data ** 3
        + 2.91583e-5 * humidity_data ** 3
        + 1.42721e-6 * tas_data ** 3 * humidity_data 
        + 1.97483e-7 * tas_data * humidity_data ** 3
        - 2.18429e-8 * tas_data ** 3 * humidity_data ** 2
        + 8.43296e-10 * tas_data ** 2 * humidity_data ** 3
        - 4.81975e-11 * tas_data ** 3 * humidity_data ** 3
        + 0.5
    )

    # Apply correction factor
    correction_mask = tas_data < 75
    heat_index_9_data[correction_mask] = tas_data[correction_mask]

    # Create a new cube with the heat index data
    heat_index_cube = location_tas_cube.copy()
    heat_index_cube.data = (heat_index_9_data - 32) * 5/9
    heat_index_cube.long_name = 'Heat Index 9'
    
    return heat_index_cube

# Function to calculate Heat Index (HI) using algorithm 10 above
def heat_index_10(location_name, location_tas_cube, location_rh_cube):
    """
    Returns the Blazejczyk et al. (2012) heat index from a tas and h cube.
    
    Parameters
    ----------
    location_tas_cube :
        An instance of `iris.cube.Cube` representing temperature data in ˚C
    location_rh_cube :
        An instance of `iris.cube.Cube` representing humidity data in %
    """
    assert location_tas_cube.units == 'celsius', "Temperature units must be in celsius"
    assert location_rh_cube.units == '%', 'Ensure humidity units are in %'

    tas_cube_copy = location_tas_cube.copy()  # Create a copy of the temperature cube

    tas_data = tas_cube_copy.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time
    humidity_data = location_rh_cube.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time

    heat_index_10_data = (
        -8.784695
        + 1.161139411 * tas_data
        + 2.338549 * humidity_data
        - 0.14611605 * tas_data * humidity_data
        - 1.2308094e-2 * tas_data ** 2
        - 1.6424828e-2 * humidity_data ** 2
        + 2.211732e-3 * tas_data ** 2 * humidity_data
        + 7.2546e-4 * tas_data * humidity_data ** 2
        - 3.582e-6 * tas_data ** 2 * humidity_data ** 2
    )

    # Apply correction factor
    correction_mask = tas_data <= 20
    heat_index_10_data[correction_mask] = tas_data[correction_mask]

    # Create a new cube with the heat index data
    heat_index_cube = location_tas_cube.copy()
    heat_index_cube.data = heat_index_10_data
    heat_index_cube.long_name = 'Heat Index 10'
    
    return heat_index_cube

# Function to calculate Heat Index (HI) using algorithm 11 above
def heat_index_11(location_name, location_tas_cube, location_rh_cube):
    """
    Returns the Patricola and Cook (2010) heat index from a tas and h cube.
    
    Parameters
    ----------
    location_tas_cube :
        An instance of `iris.cube.Cube` representing temperature data in ˚C
    location_rh_cube :
        An instance of `iris.cube.Cube` representing humidity data in %
    """
    assert location_tas_cube.units == 'celsius', "Temperature units must be in celsius"
    assert location_rh_cube.units == '%', 'Ensure humidity units are in %'

    tas_cube_copy = location_tas_cube.copy()  # Create a copy of the temperature cube
    tas_cube_copy.convert_units('Fahrenheit') # Convert temperature cube to Fahrenheit

    tas_data = tas_cube_copy.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time
    humidity_data = location_rh_cube.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time

    heat_index_11_data = (
        -42.4
        + 2.05 * tas_data
        + 10.1 * humidity_data
        - 0.255 * tas_data * humidity_data
        - (6.84e-3) * tas_data ** 2
        - (5.48e-2) * humidity_data ** 2
        + (1.23e-3) * tas_data ** 2 * humidity_data
        + (8.53e-4) * tas_data * humidity_data ** 2
        - (1.99e-6) * tas_data ** 2 * humidity_data ** 2
    )

    # Apply correction factor
    
    correction_mask = np.logical_or(tas_data <= 80, humidity_data <= 40)
    heat_index_11_data[correction_mask] = tas_data[correction_mask]

    # Create a new cube with the heat index data
    heat_index_cube = location_tas_cube.copy()
    heat_index_cube.data = (heat_index_11_data -32) * 5/9
    heat_index_cube.long_name = 'Heat Index 11'

    return heat_index_cube

# Function to calculate Heat Index (HI) using algorithm 12 above
def heat_index_12(location_name, location_tas_cube, location_td_cube):
    """
    Returns the Smoyer-Tomic and Rainham (2001) heat index from a tas and td cube.
    
    Parameters
    ----------
    location_tas_cube :
        An instance of `iris.cube.Cube` representing temperature data in ˚C
    location_td_cube :
        An instance of `iris.cube.Cube` representing dew point temperature data in ˚C
    """
    assert location_tas_cube.units == 'celsius' and location_td_cube.units == 'celsius', "Ensure cube units are in celsius"

    tas_cube = location_tas_cube.copy()
    tas_data = tas_cube.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time
    
    td_cube = location_td_cube.copy()
    dc_data = td_cube.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time

    heat_index_12_data = (
        -2.719
        + 0.994 * tas_data
        + 0.016 * dc_data ** 2
    )

    # Apply correction factor
    correction_mask = tas_data < 25
    heat_index_12_data[correction_mask] = tas_data[correction_mask]

    # Create a new cube with the heat index data
    heat_index_cube = location_tas_cube.copy()
    heat_index_cube.data = heat_index_12_data
    heat_index_cube.long_name = 'Heat Index 12'

    return heat_index_cube

# Function to calculate Heat Index (HI) using algorithm 13 above
def heat_index_13(location_name, location_tas_cube, location_td_cube):
    """
    Returns the O'Neill et al. (2003) heat index from a tas and td cube.
    
    Parameters
    ----------
    location_tas_cube :
        An instance of `iris.cube.Cube` representing temperature data in ˚C
    location_td_cube :
        An instance of `iris.cube.Cube` representing dew point temperature data in ˚C
    """
    assert location_tas_cube.units == 'celsius', "Temperature units must be in celsius"

    tas_cube_copy = location_tas_cube.copy()  # Create a copy of the temperature cube
    tas_data = tas_cube_copy.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time
    td_cube_copy = location_td_cube.copy()  # Create a copy of the temperature cube
    dc_data = td_cube_copy.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time

    heat_index_13_data = (
        -2.653
        + 0.994 * tas_data
        + 0.0153 * dc_data ** 2
    )

    # Create a new cube with the heat index data
    heat_index_cube = location_tas_cube.copy()
    heat_index_cube.data = heat_index_13_data
    heat_index_cube.long_name = 'Heat Index 13'

    return heat_index_cube

# Function to calculate Heat Index (HI) using algorithm 14 above
def heat_index_14(location_name, location_tas_cube, location_td_cube):
    """Returns the Perry et al. (2011) heat index from a tas and td cube.
    
    Parameters
    ----------
    location_tas_cube :
        An instance of :class:`iris.cube.Cube`
    location_td_cube :
        An instance of :class:`iris.cube.Cube`
    """
    assert location_tas_cube.units == 'celsius' and location_td_cube.units == 'celsius', "Ensure cube units are in celsius"

    tas_cube_copy = location_tas_cube.copy()  # Create a copy of the temperature cube
    td_cube_copy = location_td_cube.copy()  # Create a copy of the td cube

    heat_index_14_data = (-2.719 
                          + 0.994 * tas_cube_copy.data 
                          + 0.016 * td_cube_copy.data ** 2)

    heat_index_cube = tas_cube_copy.copy()  # Create a new cube with the heat index data
    heat_index_cube.data = heat_index_14_data
    heat_index_cube.long_name = 'Heat Index 14'

    return heat_index_cube

# Function to calculate Heat Index (HI) using algorithm 15 above
def heat_index_15(location_name, location_tas_cube, location_rh_cube):
    """
    Returns the Tam et al. (2008) heat index from a tas and h cube.
    
    Parameters
    ----------
    location_tas_cube :
        An instance of `iris.cube.Cube` representing temperature data in ˚C
    location_rh_cube :
        An instance of `iris.cube.Cube` representing humidity data in %
    """
    assert location_tas_cube.units == 'celsius', "Temperature units must be in celsius"
    assert location_rh_cube.units == '%', 'Ensure humidity units are in %'

    tas_cube_fahrenheit = location_tas_cube.copy()
    tas_cube_fahrenheit.convert_units('Fahrenheit') # Convert temperature cube to Fahrenheit
    
    tas_data = tas_cube_fahrenheit.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time
    humidity_data = location_rh_cube.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time

    heat_index_15_data = (
        -42.379
        + 2.049015 * tas_data
        + 10.1433 * humidity_data
        - 0.2248 * tas_data * humidity_data
        - (6.83783e-3) * tas_data ** 2
        - (5.4817e-2) * humidity_data ** 2
        + (1.229e-3) * tas_data ** 2 * humidity_data
        + (8.528e-4) * tas_data * humidity_data ** 2
        - (1.99e-6) * tas_data ** 2 * humidity_data ** 2
    )

    # Apply correction factor
    correction_mask = tas_data < 57
    heat_index_15_data[correction_mask] = tas_data[correction_mask]

    # Create a new cube with the heat index data
    heat_index_cube = location_tas_cube.copy()
    heat_index_cube.data = (heat_index_15_data -32) * 5/9
    heat_index_cube.long_name = 'Heat Index 15'

    return heat_index_cube

# Function to calculate Heat Index (HI) using algorithm 16 above
def heat_index_16(location_name, location_tas_cube, location_rh_cube):
    """
    Returns the Rothfusz (1990) heat index from a tas and h cube.
    
    Parameters
    ----------
    location_tas_cube :
        An instance of `iris.cube.Cube` representing temperature data in ˚C
    location_rh_cube :
        An instance of `iris.cube.Cube` representing humidity data in %
    """
    assert location_tas_cube.units == 'celsius', "Temperature units must be in celsius"
    assert location_rh_cube.units == '%', 'Ensure humidity units are in %'

    tas_cube_fahrenheit = location_tas_cube.copy()
    tas_cube_fahrenheit.convert_units('Fahrenheit') # Convert temperature cube to Fahrenheit
    
    tas_data = tas_cube_fahrenheit.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time
    humidity_data = location_rh_cube.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time

    heat_index_16_data = (
        -42.379
        + 2.04901523 * tas_data
        + 10.14333127 * humidity_data
        - 0.22475541 * tas_data * humidity_data
        - (6.83783e-3) * tas_data ** 2
        - (5.481717e-2) * humidity_data ** 2
        + (1.22874e-3) * tas_data ** 2 * humidity_data
        + (8.5282e-4) * tas_data * humidity_data ** 2
        - (1.99e-6) * tas_data ** 2 * humidity_data ** 2
    )

    # Create a new cube with the heat index data
    heat_index_cube = location_tas_cube.copy()
    heat_index_cube.data = (heat_index_16_data -32) * 5/9
    heat_index_cube.long_name = 'Heat Index 16'

    return heat_index_cube

# Function to calculate Heat Index (HI) using algorithm 17 above
def heat_index_17(location_name, location_tas_cube, location_rh_cube):
    """
    Returns the Fischer and Schär (2010) heat index from a tas and h cube.
    
    Parameters
    ----------
    location_tas_cube :
        An instance of `iris.cube.Cube` representing temperature data in ˚C
    location_rh_cube :
        An instance of `iris.cube.Cube` representing humidity data in %
    """
    assert location_tas_cube.units == 'celsius', "Temperature units must be in celsius"
    assert location_rh_cube.units == '%', 'Ensure humidity units are in %'

    tas_cube_copy = location_tas_cube.copy()  # Create a copy of the temperature cube

    tas_data = tas_cube_copy.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time
    humidity_data = location_rh_cube.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time

    heat_index_17_data = (
        -8.7847 
        + 1.6114 * tas_data 
        - 0.012308 * tas_data ** 2
        + (humidity_data * (2.3385 - 0.14612 * tas_data 
                           + 2.2117e-3 * tas_data ** 2))
        + (humidity_data ** 2 * (-0.016425 + 7.2546e-4 * tas_data))
        + (-3.582e-6) * tas_data ** 2)
    
    # Create a new cube with the heat index data
    heat_index_cube = location_tas_cube.copy()
    heat_index_cube.data = heat_index_17_data
    heat_index_cube.long_name = 'Heat Index 17'

    return heat_index_cube

# Function to calculate Heat Index (HI) using algorithm 18 above
def heat_index_18(location_name, location_tas_cube, location_rh_cube):
    """
    Returns the Costanzo et al. (2006) heat index from a tas and h cube.
    
    Parameters
    ----------
    location_tas_cube :
        An instance of `iris.cube.Cube` representing temperature data in ˚C
    location_rh_cube :
        An instance of `iris.cube.Cube` representing humidity data in %
    """
    assert location_tas_cube.units == 'celsius', "Temperature units must be in celsius"
    assert location_rh_cube.units == '%', 'Ensure humidity units are in %'

    tas_cube_copy = location_tas_cube.copy()  # Create a copy of the temperature cube

    tas_data = tas_cube_copy.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time
    humidity_data = location_rh_cube.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time

    heat_index_18_data = (tas_data 
                          - 0.55 * (1 - 0.001 * humidity_data) 
                          * (tas_data - 14.5))
    
    # Create a new cube with the heat index data
    heat_index_cube = location_tas_cube.copy()
    heat_index_cube.data = heat_index_18_data
    heat_index_cube.long_name = 'Heat Index 18'

    return heat_index_cube

# Function to calculate Heat Index (HI) using algorithm 19 above
def heat_index_19(location_name, location_tas_cube, location_td_cube):
    """
    Returns the Smoyer (1998) heat index from a tas and td cube.
    
    Parameters
    ----------
    location_tas_cube :
        An instance of `iris.cube.Cube` representing temperature data in ˚C
    location_td_cube :
        An instance of `iris.cube.Cube` representing dew point temperature data in ˚C
    """
    assert location_tas_cube.units == 'celsius' and location_td_cube.units == 'celsius', "Ensure cube units are in celsius"
    
    tas_cube_copy = location_tas_cube.copy()  # Create a copy of the temperature cube
    tas_data = tas_cube_copy.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time
    dc_data = location_td_cube.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time

    heat_index_19_data = (2.719 
                          + (0.994 * tas_data) 
                          + (0.016 * (dc_data ** 2)))
    
    # Create a new cube with the heat index data
    heat_index_cube = location_tas_cube.copy()
    heat_index_cube.data = heat_index_19_data
    heat_index_cube.long_name = 'Heat Index 19'

    return heat_index_cube

# Function to calculate Heat Index (HI) using algorithm 20 above
def heat_index_20(location_name, location_tas_cube, location_rh_cube):
    """
    Returns the Lajinian et al. (1997) heat index from a tas and h cube.
    
    Parameters
    ----------
    location_tas_cube :
        An instance of `iris.cube.Cube` representing temperature data in ˚C
    location_rh_cube :
        An instance of `iris.cube.Cube` representing humidity data in %
    """
    assert location_tas_cube.units == 'celsius', "Temperature units must be in celsius"
    assert location_rh_cube.units == '%', 'Ensure humidity units are in %'

    tas_cube_fahrenheit = location_tas_cube.copy()
    tas_cube_fahrenheit.convert_units('Fahrenheit') # Convert temperature cube to Fahrenheit
    
    tas_data = tas_cube_fahrenheit.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time
    humidity_data = location_rh_cube.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time

    heat_index_20_data = (tas_data
                          - ((0.55 - 0.55 * (humidity_data / 100)) 
                             * tas_data - 58)
    )

    # Create a new cube with the heat index data
    heat_index_cube = location_tas_cube.copy()
    heat_index_cube.data = (heat_index_20_data -32) * 5/9
    heat_index_cube.long_name = 'Heat Index 20'

    return heat_index_cube

# Function to calculate Heat Index (HI) using algorithm 21 above
def heat_index_21(location_name, location_tas_cube, location_td_cube):
    """
    Returns the Basara et al. (2010) heat index from a tas and td cube.
    
    Parameters
    ----------
    location_tas_cube :
        An instance of `iris.cube.Cube` representing temperature data in ˚C
    location_td_cube :
        An instance of `iris.cube.Cube` representing dew point temperature data in ˚C
    """
    assert location_tas_cube.units == 'celsius' and location_td_cube.units == 'celsius', "Ensure cube units are in celsius"

    tas_cube_copy = location_tas_cube.copy()  # Create a copy of the temperature cube
    tas_data = tas_cube_copy.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time
    td_data = location_td_cube.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time

    heat_index_21_data = (
        -2.653 
        + 0.994 * tas_data 
        + 0.368 * (td_data ** 2)
    )

    # Create a new cube with the heat index data
    heat_index_cube = location_tas_cube.copy()
    heat_index_cube.data = heat_index_21_data
    heat_index_cube.long_name = 'Heat Index 21'

    return heat_index_cube

In [6]:
%run Lu_and_Romps_vectorized.py

In [3]:
# Creating all the heat indices cubes in one go
def create_all_21_heat_indices_cubes():
    global heat_indices_cubelist
    """
    Returns heat index values from a tas and h cube for all algorithms.

    Parameters
    ----------
    location_tas_cube :
        An instance of `iris.cube.Cube` representing temperature data in ˚C
    location_rh_cube :
        An instance of `iris.cube.Cube` representing humidity data in %
    """
    tas_cube.convert_units('celsius')
    td_cube.convert_units('celsius')

    output_folder = "heat_index_cubes_nc_files"
    if not os.path.exists(output_folder):
        os.mkdir(output_folder)
    
    heat_index_functions = [
        heat_index_1, heat_index_2, heat_index_3, heat_index_4, heat_index_5,
        heat_index_6, heat_index_7, heat_index_8, heat_index_9, heat_index_10,
        heat_index_11, heat_index_12, heat_index_13, heat_index_14, heat_index_15,
        heat_index_16, heat_index_17, heat_index_18, heat_index_19, heat_index_20,
        heat_index_21
        ]
    
    for index, heat_index_functions in enumerate(heat_index_functions):
        if not os.path.exists(f'{output_folder}/{location_name}_heat_index_{index+1}.nc'):
            filename = f'{output_folder}/{location_name}_heat_index_{index+1}.nc'
            if index in [0, 4, 5, 6, 7, 8, 9, 10, 14, 15, 16, 17, 19]:
                heat_index_cube = heat_index_functions(location_name, tas_cube, rh_cube)
            elif index in [1, 2, 11, 12, 13, 18, 20]:
                heat_index_cube = heat_index_functions(location_name, tas_cube, td_cube)
            elif index == 3:
                heat_index_cube = heat_index_functions(location_name, tas_cube, es_cube)
            print(f'Heat index {index+1} cube saved as {filename}')

            save_cube(heat_index_cube, filename)
        else:
            pass
        

In [8]:
def heat_index_lu_and_romps_vectorized_pre_processing(tas_cube, rh_cube):
    global daily_tas_cubes, daily_rh_cubes, tas_cube_converted
    tas_cube_converted = tas_cube.copy()
    tas_cube_converted.convert_units('kelvin')
    tas_cube_converted.data = np.ma.masked_where(tas_cube_converted.data == -9999, tas_cube_converted.data)
    
    daily_tas_cubes = extract_daily_cubes(tas_cube_converted)

    rh_cube_converted = rh_cube.copy()
    rh_cube_converted.data = rh_cube_converted.data / 100
    rh_cube_converted.data = np.ma.masked_where(rh_cube_converted.data == -9999, rh_cube_converted.data)
    daily_rh_cubes = extract_daily_cubes(rh_cube_converted)
    
    # To extract data from monthly_tas_cubes
    daily_tas_cubes = [cube.data for cube in daily_tas_cubes]

    # To extract data from monthly_rh_cubes
    daily_rh_cubes = [cube.data for cube in daily_rh_cubes]

    return daily_tas_cubes, daily_rh_cubes

def heat_index_lu_and_romps_vectorized_processing(daily_tas_cubes, daily_rh_cubes):
    global final_heat_index_22_cube
    if os.path.isfile(f'heat_index_22.nc'):
            heat_index_22_cube = iris.load_cube(f'{location_name}_heat_index_22.nc')
            print('heat_index_22_cube already exists. Skipping Lu and Romps (2022) calculation.')
    else:
        print('creating heat_index_cube')
        # Using ProcessPoolExecutor to utilize multiple CPU cores more effectively
        heat_index_22_daily_arrays = []
        with ProcessPoolExecutor(max_workers=4) as executor:
            tasks = list(zip(daily_tas_cubes, daily_rh_cubes))
            calculated_data = tqdm(executor.map(heat_index_lu_and_romps_vectorized, *zip(*tasks)), total=len(daily_tas_cubes), desc="Calculating Heat Index")

            for data in calculated_data:
                heat_index_22_daily_arrays.append(data)

        # Creating a single cube from the results
        final_heat_index_22_cube = tas_cube.copy()
        final_heat_index_22_cube.data = np.concatenate(heat_index_22_daily_arrays, axis=0)
        iris.save(final_heat_index_22_cube, 'heat_index_22.nc')
        return final_heat_index_22_cube

def heat_index_lu_and_romps_vectorized_post_processing(final_heat_index_22_cube):
    final_heat_index_22_cube.long_name = 'Heat Index 22'
    iris.save(final_heat_index_22_cube, 'heat_index_22.nc')
    print('Heat index 22 cube saved as heat_index_22.nc')
    return final_heat_index_22_cube